In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import mne
from mne.datasets import eegbci
from mne import Epochs, pick_types, find_events
import os
import scipy.io as sio

In [113]:
def read_dataset(dataset,num):
    path = "dataset"+str(dataset)+"/"
    if dataset == 1:
        path += "A"
    elif dataset == 2:
        path += "S"
    elif dataset == 4:
        path += "B"
    e = []
    t = []
    for i in range(1,num+1):
        e.append(sio.loadmat(path+str(i).zfill(2)+"E.mat"))
        t.append(sio.loadmat(path+str(i).zfill(2)+"T.mat"))
    return e, t

In [114]:
e, t = read_dataset(4,9)

In [118]:
def prep_e(data):
    data1 = list(data['data'].tolist()[0][0][0][0])
    data2 = list(data['data'].tolist()[0][1][0][0])
    return data1, data2

def prep_t(data):
    data1 = list(data['data'].tolist()[0][0][0][0])
    data2 = list(data['data'].tolist()[0][1][0][0])
    data3 = list(data['data'].tolist()[0][2][0][0])
    return data1, data2, data3

In [127]:
t1 = []
t2 = []
t3 = []
for tt in t:
    tmp1, tmp2, tmp3 = prep_t(tt)
    t1.append(tmp1)
    t2.append(tmp2)
    t3.append(tmp3)

In [132]:
def features(data):
    x = data[0][:,:3]
    trial_raw = data[1].tolist()
    trial = [num for e in trial_raw for num in e]
    y = [num for e in data[2] for num in e]
    y = [x if x == 1 else 0 for x in y]
    atf = [num for e in data[5] for num in e]
    rate = data[3][0].tolist()
    
    return x,trial,y,atf,rate

In [149]:
x,trial,y,atf,rate = features(t1[7])

In [150]:
x.shape

(787729, 3)

In [151]:
trial

[55140,
 62068,
 64105,
 70527,
 75189,
 77590,
 86214,
 88501,
 90935,
 93194,
 102393,
 107107,
 109568,
 114076,
 118824,
 130366,
 135102,
 137418,
 139432,
 141478,
 147232,
 149592,
 156197,
 158487,
 165074,
 169682,
 174043,
 176249,
 178306,
 180593,
 183027,
 185286,
 187570,
 189679,
 199199,
 210916,
 215704,
 222458,
 231524,
 233570,
 237034,
 239324,
 244006,
 246252,
 250579,
 254711,
 261774,
 270398,
 279662,
 284138,
 286577,
 293752,
 305346,
 307796,
 310156,
 312360,
 319286,
 321602,
 323616,
 325662,
 329126,
 338344,
 340381,
 344776,
 362490,
 364777,
 367211,
 369470,
 373863,
 376230,
 383383,
 388305,
 390352,
 392668,
 402248,
 404452,
 406642,
 408923,
 413694,
 417754,
 421218,
 423508,
 428190,
 430436,
 432473,
 434763,
 438895,
 445958,
 450319,
 456869,
 461562,
 463846,
 473173,
 477936,
 482444,
 489530,
 491980,
 498734,
 505786,
 509846,
 513310,
 515600,
 517960,
 520282,
 524565,
 533442,
 538050,
 544617,
 548961,
 555938,
 558047,
 560414,
 5

In [152]:
atf

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [153]:
rate

[250]

In [154]:
y

[0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0]

In [185]:
len(e[0]['data'][0][0][0][0])

8

In [189]:
#0 = data, 1 = trial, 2 = label, 3 = sample rate, 4 = ?, 5 = artifact, 6 = gender, 7 = age
for i in range(len(e[0]['data'][0][0][0][0])):
    print(len(e[0]['data'][0][0][0][0][i]),'*'*99)
    print(e[0]['data'][0][0][0][0][i])

466314 ***************************************************************************************************
[[ -3.05250305  -3.02808303  -1.12332112 -47.37484737  -4.3956044
   46.3980464 ]
 [ -2.97924298  -0.63492063  -1.07448107 -47.37484737 -13.18681319
   34.18803419]
 [ -2.95482295  -0.21978022  -0.58608059 -39.56043956  -8.3028083
   33.6996337 ]
 ...
 [  3.05250305   3.63858364   5.00610501  11.23321123  -7.32600733
   12.21001221]
 [  1.97802198   3.73626374   4.61538462  10.25641026 -12.21001221
    5.37240537]
 [  1.53846154   2.22222222   3.61416361  -1.46520147 -26.37362637
    2.44200244]]
160 ***************************************************************************************************
[[ 54350]
 [ 57071]
 [ 59328]
 [ 61597]
 [ 63849]
 [ 66121]
 [ 68564]
 [ 70881]
 [ 73283]
 [ 75597]
 [ 78222]
 [ 80543]
 [ 83165]
 [ 85764]
 [ 88065]
 [ 90428]
 [ 93019]
 [ 95480]
 [ 97948]
 [100253]
 [102869]
 [105511]
 [108036]
 [110745]
 [113256]
 [115537]
 [117839]
 [120555]
 [12296